## Part 1

In [73]:
import re
from itertools import product

In [218]:
TEST_INFILE = "inputs/day_12_test_1.txt"
INFILE = "inputs/day_12_1.txt"

#with open(TEST_INFILE) as infile:
with open(INFILE) as infile:
    lines = infile.read().splitlines()

records_original = [(l.split(" ")[0], l.split(" ")[1]) for l in lines]
records_original =[([c for c in record[0]], [int(n) for n in record[1].split(",")]) for record in records_original]

In [219]:
records_original[-1]

(['?', '.', '?', '?', '?', '?', '?', '?', '?', '?', '?', '#', '?'], [1, 1, 1])

In [220]:
def make_regex(pattern):
    start   = r"(^|\.+)"
    damaged = r"(\?|#)"
    working = r"(\.+)"
    end     = r"(\.*$)"

    regex = fr"^{start}{damaged}{{{pattern[0]}}}"
    for n in pattern[1:]:
        regex += working
        regex += fr"{damaged}{{{n}}}"
    regex += end

    return regex

In [221]:
records_working = [(l.split(" ")[0], l.split(" ")[1]) for l in lines]
records_working =[([c if c != "?" else ["#", "."] for c in record[0]], make_regex([int(n) for n in record[1].split(",")])) for record in records_working]

In [222]:
records_original[-1], records_working[-1]

((['?', '.', '?', '?', '?', '?', '?', '?', '?', '?', '?', '#', '?'],
  [1, 1, 1]),
 ([['#', '.'],
   '.',
   ['#', '.'],
   ['#', '.'],
   ['#', '.'],
   ['#', '.'],
   ['#', '.'],
   ['#', '.'],
   ['#', '.'],
   ['#', '.'],
   ['#', '.'],
   '#',
   ['#', '.']],
  '^(^|\\.+)(\\?|#){1}(\\.+)(\\?|#){1}(\\.+)(\\?|#){1}(\\.*$)'))

In [223]:
def get_n_matches(record):
    pattern = record[0]
    regex = record[1]

    n_matches = 0
    for combo in product(*pattern):
        combo_str = "".join(combo)
        if re.match(regex, combo_str):
            #print(f"{combo_str} matches {regex}")
            n_matches += 1
        #else:
        #    print(f"{combo_str} does not match {regex}")

    return n_matches

In [224]:
sum(get_n_matches(r) for r in records_working)

7007

## Part 2

In [225]:
from functools import cache

In [226]:
records = [("".join((r[0]+["?"])*4+r[0]), r[1]*5) for r in records_original]

In [227]:
records[0]

('#?#???????#?.??#?#???????#?.??#?#???????#?.??#?#???????#?.??#?#???????#?.?',
 [3, 1, 2, 2, 3, 1, 2, 2, 3, 1, 2, 2, 3, 1, 2, 2, 3, 1, 2, 2])

In [228]:
@cache
def count_arrangements(pattern, rules):
    #print(f"Counting arrangements of {pattern} with rules {rules}")
    if not rules:
        return 0 if "#" in pattern else 1
    if not pattern:
        return 1 if not rules else 0

    result = 0

    # if we have a . it's operational so just add the next count
    # if we have a ? it _can_ be operational so add the next count, but then
    # the next if statement will consider if it can also be a #
    if pattern[0] in ".?":
        result += count_arrangements(pattern[1:], rules)
    if pattern[0] in "#?":
        #print(f"Checking rules[0] <= len(pattern): {rules[0]} <= {len(pattern)}")
        if (
            rules[0] <= len(pattern) # do we have enough characters left to still satisfy the needed run...
            and "." not in pattern[: rules[0]] # is it not broken by a .
            and (rules[0] == len(pattern) or pattern[rules[0]] != "#") # the character immediately after the block (if it exists) must not be a #
        ):
            result += count_arrangements(pattern[rules[0] + 1 :], rules[1:])

    return result

In [229]:
sum([count_arrangements(p, tuple(r)) for p, r in records])

3476169006222

In [ ]:
@cache
def dp_count(pattern, rules, i, completed, k_next):
    """
    number of ways to fill pattern[i:] given that we've already completed completed runs and are
    currently in the next run with length k_next
    """
    # base case
    if i == len(pattern):
        if k_next > 0 and completed < len(rules) and k_next == rules[completed] and completed + 1 == len(rules):
            return 1
        if k_next == 0 and completed == len(rules):
            return 1
        return 0

    if pattern[i] == ".":
        # end the current run if we get to the right length
        if k_next > 0 and completed < len(rules) and k_next == rules[completed]:
            return dp_count(pattern, rules, i + 1, completed + 1, 0)
        # we're between runs and stay between runs
        if k_next == 0:
            return dp_count(pattern, rules, i + 1, completed, 0)
        else:
            return 0
    elif pattern[i] == "#":
        if completed < len(rules) and k_next < rules[completed]:
            return dp_count(pattern, rules, i + 1, completed, k_next + 1)
        if completed == len(rules) or k_next == rules[completed]:
            return 0
    elif pattern[i] == "?":
        total = 0
        # try it as a .
        if k_next > 0 and completed < len(rules) and k_next == rules[completed]:
            total += dp_count(pattern, rules, i + 1, completed + 1, 0)
        if k_next == 0:
            total += dp_count(pattern, rules, i + 1, completed, 0)
        # try it as a #
        if completed < len(rules) and k_next < rules[completed]:
            total += dp_count(pattern, rules, i + 1, completed, k_next + 1)
        if completed == len(rules) or k_next == rules[completed]:
            total += 0
        return total

    print("Returning None for", (pattern, rules, i, completed, k_next))
    return None

In [252]:
count_arrangements(records[0][0], tuple(records[0][1]))

58564

In [253]:
dp_count(records[0][0], tuple(records[0][1]), 0, 0, 0)

58564

In [254]:
sum([count_arrangements(p, tuple(r)) for p, r in records])

3476169006222